In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow as tf
# tf.enable_eager_execution()
tf.disable_eager_execution()

In [3]:
import pandas as pd
import numpy as np
import random as random
import pyarrow as pa 
import pyarrow.parquet as pq
from fastavro import writer, reader, parse_schema
import collections

In [4]:
from tensorflow.python.keras.optimizers import Adam, SGD

In [5]:
import sonnet as snt

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
records = []
with open('okex_spot_ETH_BTC_60.avro', 'rb') as fo:
    for record in reader(fo):
        records.append(record)

In [8]:
data = pd.DataFrame(records)

In [9]:
df = data.copy()

In [10]:
df = (df.sort_values(by=['window_end']).set_index(['window_end'])[150:]).fillna(method="ffill")

In [11]:
df.head()

,all_close_price,all_close_quantity,all_count,all_high_price,all_high_quantity,all_low_price,all_low_quantity,all_mean_price,all_mean_quantity,all_open_price,...,sell_low_price,sell_low_quantity,sell_mean_price,sell_mean_quantity,sell_open_price,sell_open_quantity,sell_std_price,sell_std_quantity,sell_volume,sell_vwap
window_end,,,,,,,,,,,,,,,,,,,,,
1557016019999,0.02825,0.003355,17.0,0.02828,0.056520,0.02825,0.000058,0.028262,0.021548,0.02827,...,0.02825,0.000058,0.028262,0.021548,0.02827,0.000144,0.000010,0.024675,0.366322,0.028261
1557016049999,0.02824,0.003365,23.0,0.02826,0.293876,0.02823,0.000132,0.028243,0.050195,0.02825,...,0.02823,0.000132,0.028243,0.050195,0.02825,0.035443,0.000008,0.070967,1.154495,0.028245
1557016079999,0.02824,0.018901,93.0,0.02825,0.719105,0.02822,0.000934,0.028234,0.051819,0.02824,...,0.02822,0.000934,0.028234,0.051819,0.02824,0.007907,0.000010,0.106241,4.819172,0.028235
1557016109999,0.02824,0.007759,6.0,0.02824,0.372039,0.02824,0.000070,0.028240,0.126429,0.02824,...,0.02824,0.000070,0.028240,0.126429,0.02824,0.002850,0.000000,0.190265,0.758575,0.028240
1557016139999,0.02823,0.411671,14.0,0.02823,0.492622,0.02822,0.000562,0.028229,0.146739,0.02823,...,0.02822,0.000562,0.028229,0.146739,0.02823,0.056460,0.000003,0.173626,2.054350,0.028230


In [12]:
window_size=90
-
batch_num = len(df) - 92
max_leverage = 5
learning_rate=0.00028

In [13]:
Batch = collections.namedtuple(
    'Batch',
    [
        'frames',
        'next_prices',
        'current_prices',
        'decay',
        'commission',
        'max_leverage',
        'size'
    ]
)

In [16]:
def make_batch(batch_size=5, offset=0):
    frames=[]
    next_prices=[]
    current_prices=[]
    
    for i in range(batch_size):
        frames.append(df.iloc[i:i+window_size,:][features].values)
        next_prices.append(df.iloc[i+window_size+1][["all_close_price"]].values)
        current_prices.append(df.iloc[i+window_size][["all_close_price"]].values)

    return Batch(
        frames=tf.cast(frames, tf.float32),
        next_prices=tf.cast(next_prices, tf.float32),
        current_prices=tf.cast(current_prices, tf.float32),
        decay=3.47222222e-7,
        commission=0.0015,
        max_leverage=4,
        size=batch_size
    )

In [19]:
class CNNAgent(snt.AbstractModule):
    def __init__(self):
        super(CNNAgent, self).__init__(name='agent')

    def _build(self, input_):
#         last_action, env_output = input_
#         reward, _, _, (frame, instruction) = env_output

        # Convert to floats.
        frame = tf.expand_dims(input_, axis=2)

        frame /= 255
        with tf.variable_scope('convnet'):
          conv_out = frame
          for i, (num_ch, num_blocks) in enumerate([(16, 2), (32, 2), (32, 2)]):
            # Downscale.
            conv_out = snt.Conv2D(num_ch, 3, stride=1, padding='SAME')(conv_out)
            conv_out = tf.nn.pool(
                conv_out,
                window_shape=[3, 3],
                pooling_type='MAX',
                padding='SAME',
                strides=[2, 2])

            # Residual block(s).
            for j in range(num_blocks):
              with tf.variable_scope('residual_%d_%d' % (i, j)):
                block_input = conv_out
                conv_out = tf.nn.relu(conv_out)
                conv_out = snt.Conv2D(num_ch, 3, stride=1, padding='SAME')(conv_out)
                conv_out = tf.nn.relu(conv_out)
                conv_out = snt.Conv2D(num_ch, 3, stride=1, padding='SAME')(conv_out)
                conv_out += block_input

        conv_out = tf.nn.relu(conv_out)
        conv_out = snt.BatchFlatten()(conv_out)

        conv_out = snt.Linear(256)(conv_out)
        conv_out = tf.nn.relu(conv_out)

        return tf.layers.dense(
                conv_out, 
                1, 
                activation=tf.nn.tanh
            )
    
agent = CNNAgent()

In [23]:
next_prices = tf.placeholder(
    tf.float32,
    shape=[None, 1]
)

current_prices = tf.placeholder(
    tf.float32,
    shape=[None, 1]
)

frames = tf.placeholder(
    tf.float32,
    shape=[None,len(features)]
)

size = tf.placeholder(
    tf.float32,
    shape=[1]
)

commission = tf.placeholder(
    tf.float32,
    shape=[1]
)

decay = tf.placeholder(
    tf.float32,
    shape=[1]
)

framw

action = agent(frames)

# distribution of portfolio 
# in next step
future_omega = (
        next_prices * action
) /\
tf.reduce_sum(
    next_prices * action,
    axis=1
)[:, None]

w_t = future_omega[:size-1]
w_t1 = action[1:size]
mu = 1 - (tf.reduce_sum(
    tf.abs(w_t1[:, 1:]-w_t[:, 1:]),
    axis=1
)*commission + decay)    

free_rets = tf.reduce_sum(
    tf.concat(
        [
            action*(next_prices-current_prices),
            tf.ones([size,1])
        ], 
        axis=1
    ), 
    axis=1
)

rets = free_rets * ( 
    tf.concat(
        [
        tf.ones(1),
        mu
        ],
        axis=0
    )
)

# Mean returns
mean = tf.reduce_mean(rets)
log_mean = tf.reduce_mean(tf.log(rets))
log_mean_free = tf.reduce_mean(free_rets)

# Standard Deviation of returns
stdev = tf.sqrt(tf.reduce_mean(
   (rets - mean) ** 2
))

# Sharpe ratio
sharp_ratio = (mean - 1) / stdev

# Portfolio value
p_value = tf.reduce_prod(rets)

loss = -tf.reduce_mean(tf.log(rets))

train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

TypeError: Only integers, slices (`:`), ellipsis (`...`), tf.newaxis (`None`) and scalar tf.int32/tf.int64 tensors are valid indices, got <tf.Tensor 'sub_2:0' shape=(1,) dtype=float32>

In [30]:
with tf.Session() as session:
    batch = make_batch()
    
    output = session.run(
        fetches = [
            action,
            train_op
        ],
        feed_dict={
            frames: batch.frames,
            next_prices=batch.next_prices,
            current_prices=batch.current_prices,
            commission=batch.commission,
            decay=batch.decay,
            max_leverage=batch.max_leverage,
            size=batch.size
        }
    )

<tf.Tensor: id=1879, shape=(), dtype=float32, numpy=2.8610233e-07>

In [ ]:
def train():
    
        action = agent(
            batch.frame
        )
        
        # distribution of portfolio 
        # in next step
        future_omega = 0
        
        w_t = future_omega[:batch.size-1]
        w_t1 = action[1:batch.size]
        mu = 1 - (tf.reduce_sum(
            tf.abs(w_t1[:, 1:]-w_t[:, 1:]),
            axis=1
        )*batch.commission + batch.decay)
        
        
        free_rets = tf.reduce_sum(
            tf.concat(
                [
                    action*(batch.next_prices-batch.current_prices),
                    tf.ones([batch.size,1])
                ], 
                axis=1
            ), 
            axis=1
        )
        
        rets = free_rets * ( 
            tf.concat(
                [
                tf.ones(1),
                mu
                ],
                axis=0
            )
        )
        
        # Mean returns
        mean = tf.reduce_mean(rets)
        log_mean = tf.reduce_mean(tf.log(rets))
        log_mean_free = tf.reduce_mean(free_rets)
        
        # Standard Deviation of returns
        stdev = tf.sqrt(tf.reduce_mean(
           (rets - mean) ** 2
        ))
        
        # Sharpe ratio
        sharp_ratio = (mean - 1) / stdev
        
        # Portfolio value
        p_value = tf.reduce_prod(rets)
        
        loss = -tf.reduce_mean(tf.log(rets))
        
        tf.train.AdamOptimizer(learning_rate).minimize(
            loss, 
            global_step=self.__global_step
        )
        